In [1]:
import pandas as pd
import numpy as np

import json
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRegressor

In [32]:
train_data = pd.read_csv('train_modified.csv') # Important!!
print(train_data.shape)
train_data.head()

/home/tintash/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(903653, 56)


,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [3]:
#Print all the column names
print(train_data.columns.values)

['Unnamed: 0' 'channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime'
 'device.browser' 'device.browserSize' 'device.browserVersion'
 'device.deviceCategory' 'device.flashVersion' 'device.isMobile'
 'device.language' 'device.mobileDeviceBranding' 'device.mobileDeviceInfo'
 'device.mobileDeviceMarketingName' 'device.mobileDeviceModel'
 'device.mobileInputSelector' 'device.operatingSystem'
 'device.operatingSystemVersion' 'device.screenColors'
 'device.screenResolution' 'geoNetwork.city' 'geoNetwork.cityId'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.latitude'
 'geoNetwork.longitude' 'geoNetwork.metro' 'geoNetwork.networkDomain'
 'geoNetwork.networkLocation' 'geoNetwork.region'
 'geoNetwork.subContinent' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.transactionRevenue'
 'totals.visits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsC

In [33]:
relevantAttributes = ['visitNumber', 'date', 'fullVisitorId', 'channelGrouping',
#                       'device.deviceCategory',
                      'device.operatingSystem', 'device.browser',
                       'geoNetwork.continent', 'geoNetwork.country',
                      'totals.hits', 'totals.pageviews', 
                      'totals.transactionRevenue']
#                        'fullVisitorId']
#                       'year', 'month', 'weekday',
#                       'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']

train_data = train_data[relevantAttributes]

In [34]:
# #Figure out labeling for country param
sortedContinent = train_data.groupby('geoNetwork.continent')['totals.transactionRevenue'].mean()
sortedContinent = sortedContinent.sort_values(ascending=False)
continentCodeDict = {}
i = 0
for continent in sortedContinent.index:
    continentCodeDict[continent] = i
    i += 1
    
# # sortedSubContinent = train_data.groupby('geoNetwork.subContinent')['totals.transactionRevenue'].mean()
# # sortedSubContinent = sortedSubContinent.sort_values(ascending=False)
# # SubcontinentCodeDict = {}
# # i = 0
# # for subcontinent in sortedSubContinent.index:
# #     SubcontinentCodeDict[subcontinent] = i
# #     i += 1
    
sortedCountries = train_data.groupby('geoNetwork.country')['totals.transactionRevenue'].mean()
sortedCountries = sortedCountries.sort_values(ascending=False)
countryCodeDict = {}
i = 0
for country in sortedCountries.index:
    countryCodeDict[country] = i
    i += 1
    
# # sortedCities = train_data.groupby('geoNetwork.city')['totals.transactionRevenue'].mean()
# # sortedCities = sortedCities.sort_values(ascending=False)
# # cityCodeDict = {}
# # i = 0
# # for city in sortedCities.index:
# #     cityCodeDict[city] = i
# #     i += 1
    
sortedDevices = train_data.groupby('device.operatingSystem')['totals.transactionRevenue'].mean()
sortedDevices = sortedDevices.sort_values(ascending=False)
deviceCodeDict = {}
i = 0
for device in sortedDevices.index:
    deviceCodeDict[device] = i
    i += 1
    
# sortedDevicesCategory = train_data.groupby('device.deviceCategory')['totals.transactionRevenue'].mean()
# sortedDevicesCategory = sortedDevicesCategory.sort_values(ascending=False)
# deviceCategoryCodeDict = {}
# i = 0
# for deviceCategory in sortedDevicesCategory.index:
#     deviceCategoryCodeDict[deviceCategory] = i
#     i += 1
    
sortedBrowser = train_data.groupby('device.browser')['totals.transactionRevenue'].mean()
sortedBrowser = sortedBrowser.sort_values(ascending=False)
browserCodeDict = {}
i = 0
for browser in sortedBrowser.index:
    browserCodeDict[browser] = i
    i += 1

sortedChannelGrouping = train_data.groupby('channelGrouping')['totals.transactionRevenue'].mean()
sortedChannelGrouping = sortedChannelGrouping.sort_values(ascending=False)
channelGroupingCodeDict = {}
i = 0
for device in sortedChannelGrouping.index:
    channelGroupingCodeDict[device] = i
    i += 1

# print(channelGroupingCodeDict)
# print(deviceCodeDict)
# print(browserCodeDict)
# print(continentCodeDict)
# # print(SubcontinentCodeDict)
# print(deviceCategoryCodeDict)
# #print(cityCodeDict)

In [35]:
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    
    return df

train_data['totals.transactionRevenue'] = train_data['totals.transactionRevenue'].fillna(0)
train_data['totals.pageviews'] = train_data['totals.pageviews'].fillna(1)

trainData = train_data.sample(frac=0.8,random_state=200)
testData = train_data.drop(trainData.index)
test = testData

In [36]:
# # trainData['geoNetwork.subContinent'] = trainData['geoNetwork.subContinent'].transform(lambda x: SubcontinentCodeDict[x])
trainData['geoNetwork.continent'] = trainData['geoNetwork.continent'].transform(lambda x: continentCodeDict[x])
trainData['geoNetwork.country'] = trainData['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
# # trainData['geoNetwork.city'] = trainData['geoNetwork.city'].transform(lambda x: cityCodeDict[x])
trainData['device.operatingSystem'] = trainData['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
trainData['device.browser'] = trainData['device.browser'].transform(lambda x: browserCodeDict[x])
# trainData['device.deviceCategory'] = trainData['device.deviceCategory'].transform(lambda x: deviceCategoryCodeDict[x])
trainData['channelGrouping'] = trainData['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])

# #Test
# # testData['geoNetwork.subContinent'] = testData['geoNetwork.subContinent'].transform(lambda x: SubcontinentCodeDict[x])
testData['geoNetwork.continent'] = testData['geoNetwork.continent'].transform(lambda x: continentCodeDict[x])
testData['geoNetwork.country'] = testData['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
# # testData['geoNetwork.city'] = testData['geoNetwork.city'].transform(lambda x: cityCodeDict[x])
testData['device.operatingSystem'] = testData['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
testData['device.browser'] = testData['device.browser'].transform(lambda x: browserCodeDict[x])
# testData['device.deviceCategory'] = testData['device.deviceCategory'].transform(lambda x: deviceCategoryCodeDict[x])
testData['channelGrouping'] = testData['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])

In [37]:
train = add_time_features(trainData)
test = add_time_features(testData)
# Convert target feature to 'float' type.
trainData["totals.transactionRevenue"] = trainData["totals.transactionRevenue"].astype('float')
train['totals.hits'] = train['totals.hits'].astype(float)
test['totals.hits'] = test['totals.hits'].astype(float)
train['totals.pageviews'] = train['totals.pageviews'].astype(float)
test['totals.pageviews'] = test['totals.pageviews'].astype(float)

In [38]:
# train['hits.ratio'] = train['totals.hits']/train['totals.pageviews']
gp_fullVisitorId_train = train.groupby(['fullVisitorId']).agg('sum')
gp_fullVisitorId_train['fullVisitorId'] = gp_fullVisitorId_train.index
gp_fullVisitorId_train['mean_hits_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.hits'].transform('mean')
gp_fullVisitorId_train['mean_pageviews_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.pageviews'].transform('mean')
gp_fullVisitorId_train['sum_hits_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.hits'].transform('sum')
gp_fullVisitorId_train['sum_pageviews_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.pageviews'].transform('sum')
gp_fullVisitorId_train = gp_fullVisitorId_train[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
train = train.join(gp_fullVisitorId_train, on='fullVisitorId', how='inner', rsuffix='_')
train.drop(['fullVisitorId_'], axis=1, inplace=True)

# Test
# test['hits.ratio'] = test['totals.hits']/test['totals.pageviews']
gp_fullVisitorId_test = test.groupby(['fullVisitorId']).agg('sum')
gp_fullVisitorId_test['fullVisitorId'] = gp_fullVisitorId_test.index
gp_fullVisitorId_test['mean_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['totals.hits'].transform('mean')
gp_fullVisitorId_test['mean_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['totals.pageviews'].transform('mean')
gp_fullVisitorId_test['sum_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['totals.hits'].transform('sum')
gp_fullVisitorId_test['sum_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['totals.pageviews'].transform('sum')
gp_fullVisitorId_test = gp_fullVisitorId_test[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
test = test.join(gp_fullVisitorId_test, on='fullVisitorId', how='inner', rsuffix='_')
test.drop(['fullVisitorId_'], axis=1, inplace=True)

In [39]:
time_agg = train.groupby('date')['totals.transactionRevenue'].agg(['count', 'sum'])
year_agg = train.groupby('year')['totals.transactionRevenue'].agg(['sum'])
month_agg = train.groupby('month')['totals.transactionRevenue'].agg(['sum'])
day_agg = train.groupby('day')['totals.transactionRevenue'].agg(['sum'])
weekday_agg = train.groupby('weekday')['totals.transactionRevenue'].agg(['count','sum'])

In [40]:
print('TRAIN SET')
print('Rows: %s' % train.shape[0])
print('Columns: %s' % train.shape[1])
print('Features: %s' % train.columns.values)
print()
print('TEST SET')
print('Rows: %s' % test.shape[0])
print('Columns: %s' % test.shape[1])
print('Features: %s' % test.columns.values)
train.head()

TRAIN SET
Rows: 722922
Columns: 19
Features: ['visitNumber' 'date' 'fullVisitorId' 'channelGrouping'
 'device.operatingSystem' 'device.browser' 'geoNetwork.continent'
 'geoNetwork.country' 'totals.hits' 'totals.pageviews'
 'totals.transactionRevenue' 'year' 'month' 'day' 'weekday'
 'mean_hits_per_day' 'mean_pageviews_per_day' 'sum_hits_per_day'
 'sum_pageviews_per_day']

TEST SET
Rows: 180731
Columns: 19
Features: ['visitNumber' 'date' 'fullVisitorId' 'channelGrouping'
 'device.operatingSystem' 'device.browser' 'geoNetwork.continent'
 'geoNetwork.country' 'totals.hits' 'totals.pageviews'
 'totals.transactionRevenue' 'year' 'month' 'day' 'weekday'
 'mean_hits_per_day' 'mean_pageviews_per_day' 'sum_hits_per_day'
 'sum_pageviews_per_day']


,visitNumber,date,fullVisitorId,channelGrouping,device.operatingSystem,device.browser,geoNetwork.continent,geoNetwork.country,totals.hits,totals.pageviews,totals.transactionRevenue,year,month,day,weekday,mean_hits_per_day,mean_pageviews_per_day,sum_hits_per_day,sum_pageviews_per_day
548725,1,2017-05-23,3702632045024960671,3,5,4,2,14,10.0,10.0,0.0,2017,5,23,1,4.029331,3.454234,70610.0,60532.0
697375,2,2017-07-19,219064891723945627,3,1,1,2,14,1.0,1.0,0.0,2017,7,19,2,14.441776,11.830422,19843.0,16255.0
697913,1,2017-07-19,219064891723945627,3,1,1,2,14,2.0,2.0,0.0,2017,7,19,2,14.441776,11.830422,19843.0,16255.0
474524,1,2017-02-22,2389192114156853073,3,5,4,2,14,4.0,4.0,0.0,2017,2,22,2,4.261135,3.641990,77591.0,66317.0
399762,1,2016-09-09,1397628877965755203,3,4,1,5,20,1.0,1.0,0.0,2016,9,9,4,4.114117,3.511269,71923.0,61384.0


In [11]:
# categorical_features = ['device.isMobile'] #, 'month', 'weekday']
# train = pd.get_dummies(train, columns=categorical_features)
# test = pd.get_dummies(test, columns=categorical_features)


KeyError: "['device.isMobile'] not in index"

In [12]:
# # align both data sets (by outer join), to make they have the same amount of features,
# # this is required because of the mismatched categorical values in train and test sets
# train, test = train.align(test, join='outer', axis=1)

# # replace the nan values added by align for 0
# train.replace(to_replace=np.nan, value=0, inplace=True)
# test.replace(to_replace=np.nan, value=0, inplace=True)

In [13]:
# import datetime

# X_train = train[train['date']<=datetime.date(2017, 5, 31)]
# X_val = train[train['date']>datetime.date(2017, 5, 31)]
# test.columns.values

/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


array(['channelGrouping', 'date', 'visitNumber', 'device.deviceCategory',
       'device.operatingSystem', 'device.browser', 'geoNetwork.continent',
       'geoNetwork.country', 'totals.hits', 'totals.pageviews',
       'totals.transactionRevenue', 'fullVisitorId', 'day', 'hits.ratio',
       'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day',
       'sum_pageviews_per_day'], dtype=object)

In [41]:
# Get labels
Y_train = train['totals.transactionRevenue'].values
# Y_val = X_val['totals.transactionRevenue'].values
X_train = train.drop(['totals.transactionRevenue'], axis=1)
# X_val = X_val.drop(['totals.transactionRevenue'], axis=1)
Y_test = test['totals.transactionRevenue'].values
X_test = test.drop(['totals.transactionRevenue'], axis=1)
# Log transform the labels
Y_train = np.log1p(Y_train)
# Y_val = np.log1p(Y_val)
Y_test = np.log1p(Y_test)


In [42]:
X_test.columns.values

array(['visitNumber', 'date', 'fullVisitorId', 'channelGrouping',
       'device.operatingSystem', 'device.browser', 'geoNetwork.continent',
       'geoNetwork.country', 'totals.hits', 'totals.pageviews', 'year',
       'month', 'day', 'weekday', 'mean_hits_per_day',
       'mean_pageviews_per_day', 'sum_hits_per_day',
       'sum_pageviews_per_day'], dtype=object)

In [43]:
reduce_features = ['date', 'fullVisitorId', 'day']#, 'year', 'month', 'day','weekday', 'totals.hits'] #, 'geoNetwork.region', 
                    #'trafficSource.medium']
X_train = X_train.drop(reduce_features, axis=1)
# X_val = X_val.drop(reduce_features, axis=1)
X_test = X_test.drop(reduce_features, axis=1)

In [44]:
# normalized_features = [ 'channelGrouping',
#        'device.operatingSystem', 'device.browser', 'geoNetwork.continent',
#        'geoNetwork.country', 'year',
#        'month', 'weekday', 'mean_hits_per_day',
#        'mean_pageviews_per_day', 'sum_hits_per_day',
#        'sum_pageviews_per_day']

# # Normalize using Min-Max scaling
# scaler = StandardScaler()
# X_train[normalized_features] = scaler.fit_transform(X_train[normalized_features])
# # X_val[normalized_features] = scaler.transform(X_val[normalized_features])
# test[normalized_features] = scaler.transform(test[normalized_features])

In [45]:
#Initialize LGBM
reg = LGBMRegressor(objective = 'regression', 
                     boosting_type = 'gbdt', 
                     metric = 'rmse',
                     n_estimators = 20000, #10000
                     num_leaves = 36,
                     learning_rate = 0.03, #0.01
                     bagging_fraction = 0.9,#0.8
                     feature_fraction = 0.6,#.3
                     bagging_seed = 42,
                     max_depth = 12, #-1 
#                      categorical_feature = [train_x[cat_cols].columns.get_loc(c) for c in cat_cols ] ,
                     #cat_smooth = 20
                    )

reg.fit(
    X_train, Y_train,
    eval_set=[(X_test, Y_test)],
    early_stopping_rounds=300,
    verbose=100,
    eval_metric='rmse'
)

Training until validation scores don't improve for 300 rounds.
[100]	valid_0's rmse: 1.96789
[200]	valid_0's rmse: 1.98401
[300]	valid_0's rmse: 1.98531
Early stopping, best iteration is:
[70]	valid_0's rmse: 1.9605


LGBMRegressor(bagging_fraction=0.9, bagging_seed=42, boosting_type='gbdt',
       class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
       importance_type='split', learning_rate=0.03, max_depth=12,
       metric='rmse', min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=20000, n_jobs=-1, num_leaves=36,
       objective='regression', random_state=None, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [125]:
#Prediction
y_pred=reg.predict(X_test , num_iteration=reg.best_iteration_)

In [126]:
y = Y_test

# y_pred = y_pred + 1
# y = y + 1

# # for i in range(len(y)):
# #     print(y[i], y_pred[i])

# y_pred_ln = np.log(y_pred)
# y_true_ln = np.log(y)

difference = np.subtract(y_pred, y)

sqr = np.square(difference)

sumSqr = sqr.sum()

sumSqrMean = sumSqr/len(y)

sumSqrRootMean = np.sqrt(sumSqrMean)
sumSqrRootMean

1.6351694590041188

In [15]:
# print (X_test['totals.tr'])

In [23]:
# test_data = pd.read_csv('test_flat.csv', dtype={'fullVisitorId': 'str', 'visitId': 'str'}) # Important!!

# test_data['pageviews'] = test_data['pageviews'].fillna(1)

# X = test_data[['channelGrouping', 'date', 'visitNumber', 
#                       'deviceCategory', 'isMobile', 'operatingSystem', 'browser',
#                       'city', 'continent', 'country', 'region', 
#                       'subContinent',
#                       'hits', 'pageviews',
#                       'medium', 'fullVisitorId']]

# X['subContinent'] = X['subContinent'].transform(lambda x: SubcontinentCodeDict[x])
# X['continent'] = X['continent'].transform(lambda x: continentCodeDict[x])
# X['country'] = X['country'].transform(lambda x: countryCodeDict.get(x,220))
# X['city'] = X['city'].transform(lambda x: cityCodeDict.get(x, (len(test_data['city']) -2)))
# X['operatingSystem'] = X['operatingSystem'].transform(lambda x: deviceCodeDict.get(x,20))
# X['browser'] = X['browser'].transform(lambda x: browserCodeDict.get(x, (len(test_data['browser'])-2)))
# X['deviceCategory'] = X['deviceCategory'].transform(lambda x: deviceCategoryCodeDict[x])
# X['channelGrouping'] = X['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])
# test = add_time_features(X)

# gp_fullVisitorId_test = test.groupby(['fullVisitorId']).agg('sum')
# gp_fullVisitorId_test['fullVisitorId'] = gp_fullVisitorId_test.index
# gp_fullVisitorId_test['mean_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['hits'].transform('mean')
# gp_fullVisitorId_test['mean_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['pageviews'].transform('mean')
# gp_fullVisitorId_test['sum_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['hits'].transform('sum')
# gp_fullVisitorId_test['sum_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['pageviews'].transform('sum')
# gp_fullVisitorId_test = gp_fullVisitorId_test[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
# test = test.join(gp_fullVisitorId_test, on='fullVisitorId', how='inner', rsuffix='_')
# test.drop(['fullVisitorId_'], axis=1, inplace=True)

# categorical_features = ['isMobile'] #, 'month', 'weekday']
# test = pd.get_dummies(test, columns=categorical_features)

# reduce_features = ['year', 'month', 'day','weekday', 'date', 'fullVisitorId', 'region', 
#                     'medium']
# test = test.drop(reduce_features, axis=1)

# y_pred = reg.predict(test, num_iteration=reg.best_iteration_)

# y_pred += 1

# test_data['predicted'] = y_pred
# # test_data[['fullVisitorId', 'predicted']].to_csv('output_row.csv')

# y_pred_sum = test_data.groupby('fullVisitorId')['predicted'].sum()

# y_pred_log = np.log(y_pred_sum)
        
# #y_pred_log.to_csv('output_log.csv')

# print(y_pred_log)

# # y_pred_sum.to_csv('output_sum.csv')

In [24]:
# submission = pd.DataFrame({'fullVisitorId':y_pred_log.index, 'PredictedLogRevenue':y_pred_log.values})
# submission.to_csv("submission.csv", index=False)

In [25]:
# test.columns.values

In [26]:
# test_data.shape